imports

In [17]:
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neural_network import MLPClassifier

get data from csv


In [18]:
full_data=pd.read_csv('Airplane.csv')

clean data

In [19]:
satisfaction_mapping = { "satisfied": 1,"neutral or dissatisfied": 0}
full_data["satisfaction"] = full_data["satisfaction"].apply(lambda x: satisfaction_mapping[x]).astype(int)

Customer_mapping = { "Loyal Customer": 1,"disloyal Customer": 0}
full_data["Customer Type"] = full_data["Customer Type"].apply(lambda x: Customer_mapping[x]).astype(int)

Gender_mapping = { "Male": 1,"Female": 0}
full_data["Gender"] = full_data["Gender"].apply(lambda x : Gender_mapping[x]).astype(int)

class_mapping = {"Eco": 0, "Business": 1, "Eco Plus": 2}
full_data["Class"] = full_data["Class"].apply(lambda x : class_mapping[x]).astype(int)

Traveltype_mapping = {"Personal Travel": 1,"Business travel": 0}
full_data["Type of Travel"] = full_data["Type of Travel"].apply(lambda x : Traveltype_mapping[x]).astype(int);

full_data['Age'], bins = pd.qcut(full_data['Age'],q = 8,labels=False, retbins=True)

#-------------------------------------------------------
max_flight_distance = full_data["Flight Distance"].max()
min_flight_distance = full_data["Flight Distance"].min()

num_bins_flight_distance = 8

full_data['Flight Distance'] = pd.cut(
    full_data['Flight Distance'], bins=num_bins_flight_distance, labels=False, right=False)

full_data.loc[full_data['Flight Distance'] < min_flight_distance, 'Flight Distance'] = 0
bin_size_flight_distance = (max_flight_distance - min_flight_distance) / num_bins_flight_distance

for i in range(1, num_bins_flight_distance + 1):
    lower_bound_flight_distance = (bin_size_flight_distance * (i-1)) + min_flight_distance
    upper_bound_flight_distance = (bin_size_flight_distance * i) + min_flight_distance
    
    full_data.loc[(full_data["Flight Distance"] < upper_bound_flight_distance) &
                  (full_data["Flight Distance"] >= lower_bound_flight_distance), 'Flight Distance'] = i

full_data.loc[full_data['Flight Distance'] > max_flight_distance, 'Flight Distance'] = num_bins_flight_distance + 1
#------------------------------------------------------------------

#------------------------------------------------------------------
max_departure_delay = full_data["Departure Delay in Minutes"].max()
min_departure_delay = full_data["Departure Delay in Minutes"].min()
num_bins_departure = 8

full_data['Departure Delay in Minutes'] = pd.cut(
    full_data['Departure Delay in Minutes'], bins=num_bins_departure, labels=False, right=False)

full_data.loc[full_data['Departure Delay in Minutes'] < min_departure_delay, 'Departure Delay in Minutes'] = 0
bin_size_departure = (max_departure_delay - min_departure_delay) / num_bins_departure

for i in range(1, num_bins_departure + 1):
    lower_bound_departure = (bin_size_departure * (i-1)) + min_departure_delay
    upper_bound_departure = (bin_size_departure * i) + min_departure_delay
    
    full_data.loc[(full_data["Departure Delay in Minutes"] < upper_bound_departure) &
                  (full_data["Departure Delay in Minutes"] >= lower_bound_departure), 'Departure Delay in Minutes'] = i

full_data.loc[full_data['Departure Delay in Minutes'] > max_departure_delay, 'Departure Delay in Minutes'] = num_bins_departure + 1
#------------------------------------------------------

#------------------------------------------------------
max_delay = full_data["Arrival Delay in Minutes"].max()
min_delay = full_data["Arrival Delay in Minutes"].min()
num_ranges = 4

range_size = (max_delay - min_delay) / num_ranges
full_data.loc[full_data['Arrival Delay in Minutes'] < min_delay, 'Arrival Delay in Minutes'] = 0

for i in range(1, num_ranges+1):
    lower_bound = (range_size * (i-1)) + min_delay
    upper_bound = (range_size * i) + min_delay
    
    full_data.loc[(full_data["Arrival Delay in Minutes"] < upper_bound) &
                  (full_data["Arrival Delay in Minutes"] >= lower_bound), 'Arrival Delay in Minutes'] = i

full_data.loc[full_data['Arrival Delay in Minutes'] > max_delay, 'Arrival Delay in Minutes'] = num_ranges + 1
full_data["Arrival Delay in Minutes"].fillna(full_data["Arrival Delay in Minutes"].median(), inplace=True)

select train and test data

In [20]:
x=full_data[['Gender','Customer Type','Age','Type of Travel','Class','Flight Distance','Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink','Online boarding','Seat comfort','Inflight entertainment','On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness','Departure Delay in Minutes','Arrival Delay in Minutes']]
y=full_data[['satisfaction']]
X_Train=x.iloc[:85000,:]
Y_Train=y.iloc[:85000,:]
X_Test=x.iloc[85000:,:]
Y_Test=y.iloc[85000:,:]

In [32]:
clf=MLPClassifier(hidden_layer_sizes=(10,5),learning_rate_init=0.02)

clf.fit(X_Train,Y_Train.values.ravel())
ypred=clf.predict(X_Test)
accuracy = (accuracy_score(ypred,Y_Test))
print(accuracy)

0.9331887431231486
